In [38]:
from patent_client_plus import PrivatePair

apps = """16274906
16265808
16259910
16211847
16182206
16164456
16109233
16040132
16028047
15962607
15806826
15697572
15672790
15654156
15588608
15218457
15189090
14922969
13999329
13893195
13373223
13245496
13194877
13134819""".split('\n')

app_objs = [PrivatePair.objects.get(a) for a in apps]
app

<PrivatePair appl_id=15654156>

In [41]:
fh = list()
for app in app_objs:
    fh += [{**fh_entry, "appl_id": app.appl_id} for fh_entry in app.file_history]
len(fh)

1631

In [50]:
[
    {'appl_id': app.appl_id,
     'oas': len(tuple(f for f in app.file_history if f['code'] in ['CTFR', 'CTNF'])),
     'noas': [f['date'] for f in app.file_history if f['code'] == "NOA"],
    }
    for app in app_objs
]

[{'appl_id': '16274906', 'oas': 0, 'noas': []},
 {'appl_id': '16265808', 'oas': 0, 'noas': []},
 {'appl_id': '16259910', 'oas': 0, 'noas': []},
 {'appl_id': '16211847', 'oas': 0, 'noas': []},
 {'appl_id': '16182206', 'oas': 0, 'noas': []},
 {'appl_id': '16164456', 'oas': 0, 'noas': []},
 {'appl_id': '16109233', 'oas': 1, 'noas': [datetime.date(2019, 7, 15)]},
 {'appl_id': '16040132', 'oas': 0, 'noas': []},
 {'appl_id': '16028047', 'oas': 0, 'noas': []},
 {'appl_id': '15962607', 'oas': 0, 'noas': []},
 {'appl_id': '15806826', 'oas': 1, 'noas': []},
 {'appl_id': '15697572', 'oas': 3, 'noas': [datetime.date(2019, 5, 2)]},
 {'appl_id': '15672790', 'oas': 0, 'noas': []},
 {'appl_id': '15654156', 'oas': 1, 'noas': []},
 {'appl_id': '15588608',
  'oas': 0,
  'noas': [datetime.date(2017, 11, 21), datetime.date(2017, 9, 14)]},
 {'appl_id': '15218457', 'oas': 1, 'noas': [datetime.date(2018, 9, 10)]},
 {'appl_id': '15189090', 'oas': 2, 'noas': [datetime.date(2019, 4, 5)]},
 {'appl_id': '14922969'

In [44]:
kinds = set((f['code'], f['description']) for f in fh)
kinds

{('136A', 'Authorization for Extension of Time all replies'),
 ('1449', 'List of References cited by applicant and considered by examiner'),
 ('892', 'List of references cited by examiner'),
 ('A...', 'Amendment/Req. Reconsideration-After Non-Final Reject'),
 ('A.I.', 'Informal or Non-Responsive Amendment'),
 ('A.LA', 'Untimely (Late) Amendment Filed'),
 ('A.NA', 'Amendment after Notice of Allowance (Rule 312)'),
 ('A.NE', 'Response After Final Action'),
 ('A.NE.AFCP', 'After Final Consideration Program Request'),
 ('A.NE.AFCP.D', 'After Final Consideration Program Decision'),
 ('A.NQ', 'Amendment Crossed in Mail'),
 ('A.PE', 'Preliminary Amendment'),
 ('ABN', 'Abandonment'),
 ('ABST', 'Abstract'),
 ('ADS', 'Application Data Sheet'),
 ('AMSB', 'Amendment Submitted/Entered with Filing of CPA/RCE'),
 ('ANE.I',
  'Amendment After Final or under 37CFR 1.312, initialed by the examiner.'),
 ('APP.FILE.REC', 'Filing Receipt'),
 ('BIB', 'Bibliographic Data Sheet'),
 ('CFILE', 'Request for Corr

In [ ]:

deadline_codes = ['CTNF', 'NTC.MSS.PRT']



In [17]:
import datetime
from enum import IntEnum

Weekdays = IntEnum('Weekdays', 'mon tue wed thu fri sat sun', start=0)
import holidays

us_holidays = holidays.UnitedStates()

def next_business_day(date):
    while Weekdays.sat <= date.weekday() <= Weekdays.sun or date in us_holidays:
        date += datetime.timedelta(days=1)
    return date


datetime.date(2019, 8, 5)

In [22]:
filing_date = file_history[-1]['date']
filing_date

datetime.date(2017, 7, 19)

In [36]:
from dateutil.relativedelta import relativedelta

def ids_date(file_history):
    """
    37 C.F.R. 1.97(b):
    
    An information disclosure statement shall be considered by the Office if filed by the applicant within any one of the following time periods:

    (1) Within three months of the filing date of a national application other than a continued prosecution application under § 1.53(d);
    (2) Within three months of the date of entry of the national stage as set forth in § 1.491  in an international application;
    . . .
    (5) Within three months of the date of publication of the international registration under Hague Agreement Article 10(3) in an international design application.

    """
    return {
        "code": "IIDS",
        "name": "Initial Information Disclosure Deadline",
        "due": next_business_day(filing_date + relativedelta(months=3)),
        "authority": "37 C.F.R. 1.97(b)",
    }

def non_final_office_action_date(file_history):
    """
    37 C.F.R. 1.134:
    An Office action will notify the applicant of any non-statutory or shortened statutory time period set for reply to an Office action. 
    Unless the applicant is notified in writing that a reply is required in less than six months, a maximum period of six months is allowed.
    
    NOTE: There is no regulation requiring a specific shortened period, although 3 months is typical
    
    The exact text of the deadline reads: 
    
    A SHORTENED STATUTORY PERIOD FOR REPLY IS SET TO EXPIRE 3 MONTHS FROM THE MAILING DATE OF THIS COMMUNICATION.
    convert cover.png -crop 2100x100+140+450 - | tesseract stdin stdout
    """
    deadlines = list()
    for nfoa in (doc for doc in file_history if doc['code'] == "CTNF"):
        deadlines += [
            {
                "code": "INFOA",
                "name": "Initial Deadline to respond to Non-Final Office Action",
                "due": next_business_day(nfoa['date'] + relativedelta(months=3)),
                "authority": "NONE",
                "notes": "CONFIRM DEADLINE ON DOCUMENT"
            },
            {
                "code": "1NFOA",
                "name": "1 Month Extension of Time to respond to Non-Final Office Action",
                "due": next_business_day(nfoa['date'] + relativedelta(months=4)),
                "authority": "NONE",
                "notes": "Additional Fees Due"
            },
            {
                "code": "2NFOA",
                "name": "Initial Deadline to respond to Non-Final Office Action",
                "due": next_business_day(nfoa['date'] + relativedelta(months=5)),
                "authority": "NONE",
                "notes": "Additional Fees Due"
            },
            {
                "code": "FNFOA",
                "name": "Final Deadline to respond to Non-Final Office Action",
                "due": next_business_day(nfoa['date'] + relativedelta(months=6)),
                "authority": "37 C.F.R. 1.134"
            }]
    return deadlines

# Missing Parts: convert missing-000.png -crop 2100x125+220+1620 - | tesseract stdin stdout

scheduling_functions = [
    ids_date,
    non_final_office_action_date,
]

deadlines = [func(file_history) for func in scheduling_functions]
deadlines
        

[{'code': 'IIDS',
  'name': 'Initial Information Disclosure Deadline',
  'due': datetime.date(2017, 10, 19),
  'authority': '37 C.F.R. 1.97(b)'},
 [{'code': 'INFOA',
   'name': 'Initial Deadline to respond to Non-Final Office Action',
   'due': datetime.date(2019, 8, 2),
   'authority': 'NONE',
   'notes': 'CONFIRM DEADLINE ON DOCUMENT'},
  {'code': '1NFOA',
   'name': '1 Month Extension of Time to respond to Non-Final Office Action',
   'due': datetime.date(2019, 9, 3),
   'authority': 'NONE',
   'notes': 'Additional Fees Due'},
  {'code': '2NFOA',
   'name': 'Initial Deadline to respond to Non-Final Office Action',
   'due': datetime.date(2019, 10, 2),
   'authority': 'NONE',
   'notes': 'Additional Fees Due'},
  {'code': 'FNFOA',
   'name': 'Final Deadline to respond to Non-Final Office Action',
   'due': datetime.date(2019, 11, 4),
   'authority': '37 C.F.R. 1.134'}]]

In [30]:
file_history

[{'category': 'PROSECUTION',
  'description': 'Communication - Re:  Power of Attorney (PTOL-308)',
  'code': 'N570',
  'date': datetime.date(2019, 6, 27),
  'pages': 1,
  'selected': False},
 {'category': 'PROSECUTION',
  'description': 'Power of Attorney',
  'code': 'PA..',
  'date': datetime.date(2019, 6, 25),
  'pages': 1,
  'selected': False},
 {'category': 'PROSECUTION',
  'description': 'EFS Acknowledgment Receipt',
  'code': 'N417',
  'date': datetime.date(2019, 6, 25),
  'pages': 2,
  'selected': False},
 {'category': 'PROSECUTION',
  'description': 'Non-Final Rejection',
  'code': 'CTNF',
  'date': datetime.date(2019, 5, 2),
  'pages': 7,
  'selected': False},
 {'category': 'PROSECUTION',
  'description': 'List of references cited by examiner',
  'code': '892',
  'date': datetime.date(2019, 5, 2),
  'pages': 2,
  'selected': False},
 {'category': 'PROSECUTION',
  'description': 'Search information including classification, databases and other search related notes',
  'code': '